### Installare le dipendenze

In [ ]:
%pip install --quiet sonar-space seaborn pandas

Caricamento dei modelli Sonar

In [ ]:
import torch

from sonar.inference_pipelines.text import (
    TextToEmbeddingModelPipeline,
    TextToTextModelPipeline,
)

Inizializzazione dei modelli tramite Pipeline

In [ ]:
text2vec = TextToEmbeddingModelPipeline(
    encoder="text_sonar_basic_encoder", tokenizer="text_sonar_basic_encoder"
)
text2text = TextToTextModelPipeline(
    encoder="text_sonar_basic_encoder",
    decoder="text_sonar_basic_decoder",
    tokenizer="text_sonar_basic_encoder",
)  

 Definizione delle frasi

In [ ]:
italian_verses = [
    "Nel mezzo del cammin di nostra vita",
    "Mi ritrovai per una selva oscura",
    "Che la diritta via era smarrita.",
    "Ahi quanto a dir qual era è cosa dura",
    "Esta selva selvaggia e aspra e forte",
    "Che nel pensier rinova la paura!",
    "Tant'era amara che poco è più morte",
    "Ma per trattar del ben ch'i' vi trovai",
    "Dirò de l'altre cose ch'i' v'ho scorte."
]

english_verses = [
    "Midway upon the journey of our life",
    "I found myself within a forest dark",
    "For the straightforward pathway had been lost.",
    "Ah me! how hard a thing it is to say",
    "What was this forest savage, rough, and stern",
    "Which in the very thought renews the fear.",
    "So bitter is it, death is little more.",
    "But of the good to treat which there I found",
    "Speak will I of the other things I saw there."
]

Ottenere embeddings per le frasi

In [ ]:
#Converte il testo italiano in un embedding vettoriale
output_ita = text2vec.predict(italian_verses, source_lang="ita_Latn")
embed_italian = torch.nn.functional.normalize(output_ita, dim=-1).numpy()

#Converte il testo inglese in un embedding vettoriale
output_eng = text2vec.predict(english_verses, source_lang="eng_Latn")
embed_english = torch.nn.functional.normalize(output_eng, dim=-1).numpy()

Calcolo della similarità tra frasi tramite il coseno e visualizzazione tramite heatmap

In [ ]:
#!pip install seaborn pandas
import pandas as pd
import seaborn as sns

#similarità tramite coseno
cos_sim = embed_italian.dot(embed_english.T)

#conversione della similarità in una tabella
sim_df = pd.DataFrame(cos_sim, columns=english_verses, index=italian_verses)

#visualizzazione similarità: Blu intenso[Alta similarità coseno (~1.0)], Bianco[Similarità neutra (~0.0)], Rosso intenso[Similarità negativa (~-1.0)]
plt.figure(figsize=(10, 8))
sns.heatmap(sim_df, cmap="vlag", annot=True, fmt=".2f", linewidths=0.5)
plt.title("Cosine Similarity: Divina Commedia (Italiano ↔ Inglese)")
plt.xlabel("Versi in Inglese")
plt.ylabel("Versi in Italiano")
plt.show()

Traduzione delle frasi con Sonar

In [ ]:
inglese_tradotto = text2text.predict(italian_verses, source_lang="ita_Latn", target_lang="eng_Latn")
italiano_tradotto = text2text.predict(english_verses, source_lang="eng_Latn", target_lang="ita_Latn")

Valutazione della qualità della traduzione con BLEU score (Bilingual Evaluation Understudy) 
Il BLEU score è compreso tra 0 e 100 (ma di solito è sotto 50). 1-10 (Traduzione molto scarsa), 10-30(Traduzione accettabile ma con molti errori), 30-50(Buona qualità, ma non perfetta), 50(Ottima qualità, vicina a una traduzione umana)

In [ ]:
import sacrebleu

# Calcolo BLEU score
bleu_ita_to_eng = sacrebleu.corpus_bleu(inglese_tradotto, [english_verses], tokenize="flores200")
bleu_eng_to_ita = sacrebleu.corpus_bleu(italiano_tradotto, [italian_verses], tokenize="flores200")

# Stampa dei risultati
print("*" * 100)
print("Italiano → Inglese BLEU score:")
print(bleu_ita_to_eng)
print("*" * 100)
print("Inglese → Italiano BLEU score:")
print(bleu_eng_to_ita)